In [5]:
import ijson.backends.yajl2_c as ijson
from shapely.geometry import shape
from multiprocess import Pool
import csv
import geopandas as gp
import numpy as np
from glob import glob
import pandas as pd
import gzip
import json

# Check verse file

In [2]:
glob("/home/nhat/regrid-bucket/verse*")

['/home/nhat/regrid-bucket/verse.csv.gz']

In [24]:
verse = pd.read_csv('/home/nhat/regrid-bucket/verse.csv.gz')
verse.shape # (3229, 25)

(3229, 25)

In [25]:
# ((3121, 25), (3121, 25))
# # Generally if there is a null table_name in verse, the county is not in our dataset.
verse_cover = verse.query('last_refresh == last_refresh | table_name == table_name')
verse.query('last_refresh == last_refresh & table_name == table_name'
           ).shape, verse.query('last_refresh == last_refresh | table_name == table_name').shape

((3121, 25), (3121, 25))

# All recent refreshes


In [50]:
ver = pd.read_csv('/home/nhat/regrid-bucket/verse.csv.gz').sort_values('last_refresh')
print(sorted(ver.dropna(subset=['last_refresh']).last_refresh.unique()))


['2016/02/01', '2016/03/31', '2016/04/04', '2016/04/05', '2016/04/27', '2016/04/28', '2016/06/03', '2016/06/12', '2016/07/11', '2018/08/31', '2018/09/20', '2018/10/17', '2018/12/05', '2019/05/07', '2019/06/03', '2019/07/16', '2019/08/01', '2019/08/06', '2019/09/24', '2019/10/01', '2019/10/21', '2019/11/11', '2019/11/13', '2019/11/25', '2019/12/18', '2020/01/20', '2020/02/03', '2020/02/11', '2020/02/17', '2020/03/04', '2020/03/09', '2020/03/11', '2020/04/13', '2020/04/27', '2020/05/11', '2020/05/12', '2020/05/26', '2020/06/08', '2020/06/22', '2020/07/06', '2020/07/20', '2020/08/03', '2020/09/14', '2020/09/28', '2020/10/13', '2020/10/26', '2020/11/09', '2020/11/23', '2020/12/07', '2020/12/17', '2021/01/19', '2021/02/01', '2021/02/15', '2021/03/01', '2021/03/16', '2021/03/31', '2021/04/13', '2021/04/27', '2021/05/11', '2021/05/25', '2021/06/09', '2021/06/21', '2021/06/22', '2021/06/30', '2021/07/07', '2021/07/20', '2021/08/03', '2021/08/18', '2021/08/31', '2021/09/14', '2021/09/28', '2021

# Create stem txt files (for parallel parsing)

In [26]:
SF57 = {'01': 'AL', '02': 'AK', '04': 'AZ', '05': 'AR', '06': 'CA', '08': 'CO', '09': 'CT', '10': 'DE', '11': 'DC', '12': 'FL', '13': 'GA', '15': 'HI', '16': 'ID', '17': 'IL', '18': 'IN', '19': 'IA', '20': 'KS', '21': 'KY', '22': 'LA', '23': 'ME', '24': 'MD', '25': 'MA', '26': 'MI', '27': 'MN', '28': 'MS', '29': 'MO', '30': 'MT', '31': 'NE', '32': 'NV', '33': 'NH', '34': 'NJ', '35': 'NM', '36': 'NY', '37': 'NC', '38': 'ND', '39': 'OH', '40': 'OK', '41': 'OR', '42': 'PA', '44': 'RI', '45': 'SC', '46': 'SD', '47': 'TN', '48': 'TX', '49': 'UT', '50': 'VT', '51': 'VA', '53': 'WA', '54': 'WV', '55': 'WI', '56': 'WY', '60': 'AS', '66': 'GU', '69': 'MP', '72': 'PR', '74': 'UM', '78': 'VI'}
SF57R = {value : key for (key, value) in SF57.items()}

# 30sec
d = {}
s_stems = []
for sf in SF57R:
    sf = sf.lower()
    listfiles = glob(f"/home/nhat/regrid-bucket/{sf}_*.ndgeojson.gz")
    d[sf] = (len(listfiles), listfiles)
    s_stems += listfiles
# list of present filename_stems in geo bucket    
s_stems = [k.split('/')[-1].split('.')[0] for k in s_stems]  



(set(), set())

In [27]:
# (set(), set()) = good!  no discrepancy
set(verse_cover.filename_stem) - set(s_stems), set(s_stems) - set(verse_cover.filename_stem)

(set(), set())

In [28]:
# (set(), {'AS', 'UM'})
set(verse_cover.state) - set(SF57R),  set(SF57R) - set(verse_cover.state) 

(set(), {'AS', 'UM'})

In [29]:
# Write stems to files
with open('stems_sf57_1.txt', 'w') as f:
    for stem in s_stems[:1600]:
        f.write(f"{stem}\n")

with open('stems_sf57_2.txt', 'w') as f:
    for stem in s_stems[1600:]:
        f.write(f"{stem}\n")

## Copy ndgeojson.gz into local folder
- gsutil -m cp gs://boss-geo-data/*.ndgeojson.gz /home/nhat/demand-points/attom/data/regrid-gz/

In [55]:
# SHOULD BE 3121 gzips
! ls data/regrid-gz/* | wc -l

3121


# RUN parallel parse-regrid.py = 1.5 hr! 
- read each gzip, then write to props (all columns, including the geom columns)


In [ ]:
bash /home/nhat/demand-points/attom/cmds.bash &

In SSH terminal: type exit (or press Ctrl+D), all background process will remain, 


In [48]:
! ls data/regrid-gz/* | wc -l
! ls data/regrid-feather/* | wc -l
! ls data/regrid-csv/* | wc -l

3121
55
3121


In [50]:
! tail -5 cmds.log

Exit status: 0
 ========= STARTING @ 2022-05-31@03:35  ========= 
 ========= COMPLETE @ 2022-05-31@04:45  ========= 
START parsing data/regrid-csv/ca_los_angeles.csv
Complete parsing #2416949 features for data/regrid-csv/ca_los_angeles.csv


In [35]:
# # Remove results from failed attemps
# ! rm data/regrid-csv/*
# ! rm cmds.log

rm: cannot remove 'data/regrid-csv/*': No such file or directory
rm: cannot remove 'cmds.log': No such file or directory


# Small steps testings

In [85]:
# read sample ndgeojson.gz with geopandas
f = '/home/nhat/regrid-bucket/ak_denali.ndgeojson.gz'
f = '/home/nhat/demand-points/attom/data/regrid-gz/ak_denali.ndgeojson.gz'
with gzip.open(f) as fin:
    den = gp.read_file(fin)
den.shape # (3127, 129)

(3127, 129)

In [22]:
# read sample ndgeojson.gz with geopandas
f = '/home/nhat/demand-points/attom/data/regrid-gz/ak_denali.ndgeojson.gz'
filename_stem = 'ak_denali'
with gzip.open(f) as fin:
    # fin = ijson.items(fin, 'features')
    # fin = fin.read() 123, 32, 34 [???]
    # chunks = pd.read_json(fin, lines=True, chunksize=10_000) 
    for i, line in zip(range(3), fin):
        line = json.loads(line)
        properties = line['properties']
        row_props = []
        for col in all_cols:
            v = str(properties.get(col, "NONE")).strip()
            if v in {"", "None", "NONE"}:
                row_props.append("")
            else:
                row_props.append(v)
        # print(i, row_props)
                
        geom = line['geometry']
        geom = shape(geom).wkt
        print(type(geom), geom)

<class 'str'> POLYGON ((-149.1977599 64.3464647, -149.1977491 64.346749, -149.1984113 64.3467491, -149.1984222 64.3464647, -149.1977599 64.3464647))
<class 'str'> POLYGON ((-149.1914749 64.3458689, -149.1914743 64.3458853, -149.1921366 64.3458854, -149.1921471 64.3456093, -149.1914849 64.3456092, -149.1914749 64.3458689))
<class 'str'> POLYGON ((-149.0451748 64.3600223, -149.0449466 64.3607001, -149.046252 64.3607001, -149.046253 64.3607001, -149.0464811 64.3600223, -149.0451748 64.3600223))


In [ ]:
# from glob import glob
# import pandas as pd
# from multiprocess import Pool


# all_props = ['geoid', 'sourceagent', 'parcelnumb', 'usecode', 'usedesc', 'zoning', 'zoning_description', 'struct', 'multistruct', 'structno', 'yearbuilt', 'numstories', 'numunits', 'structstyle', 'parvaltype', 'improvval', 'landval', 'parval', 'agval', 'saleprice', 'saledate', 'taxamt', 'owntype', 'owner', 'ownfrst', 'ownlast', 'owner2', 'owner3', 'owner4', 'subsurfown', 'subowntype', 'mailadd', 'mail_address2', 'careof', 'mail_addno', 'mail_addpref', 'mail_addstr', 'mail_addsttyp', 'mail_addstsuf', 'mail_unit', 'mail_city', 'mail_state2', 'mail_zip', 'mail_urbanization', 'mail_country', 'address', 'address2', 'original_address', 'saddno', 'saddpref', 'saddstr', 'saddsttyp', 'saddstsuf', 'sunit', 'scity', 'city', 'county', 'state2', 'szip', 'urbanization', 'location_name', 'address_source', 'legaldesc', 'plat', 'book', 'page', 'block', 'lot', 'neighborhood', 'subdivision', 'qoz', 'qoz_tract', 'census_block', 'census_blockgroup', 'census_tract', 'census_school_district', 'sourceref', 'sourcedate', 'sourceurl', 'recrdareatx', 'recrdareano', 'gisacre', 'll_gisacre', 'sqft', 'll_gissqft', 'll_bldg_footprint_sqft', 'll_bldg_count', 'reviseddate', 'path', 'll_stable_id', 'll_uuid', 'll_updated_at', 'dpv_status', 'dpv_codes', 'dpv_notes', 'dpv_type', 'cass_errorno', 'rdi', 'usps_vacancy', 'usps_vacancy_date', 'cdl_raw', 'cdl_date', 'cdl_majority_category', 'cdl_majority_percent', 'padus_public_access', 'lbcs_activity', 'lbcs_activity_desc', 'lbcs_function', 'lbcs_function_desc', 'lbcs_structure', 'lbcs_structure_desc', 'lbcs_site', 'lbcs_site_desc', 'lbcs_ownership', 'lbcs_ownership_desc', 'lat', 'lon', 'taxyear', 'stacked_flag', 'll_last_refresh', 'll_address_count', 'homestead_exemption', 'alt_parcelnumb1', 'alt_parcelnumb2', 'alt_parcelnumb3', 'parcelnumb_no_formatting', 'plss_township', 'plss_section', 'plss_range' ]
# regrid_header = all_props + ['geom']

# sf_code = "mo"

# def func(i):
#     return pd.read_csv(i, dtype=str, sep='\t',
#                  names = regrid_header, # use all columns
#                 )

    
# list_of_files = glob(f'data/regrid-csv/{sf_code}*.csv')
# with Pool(16) as pool:
#     def read_regrid_jsonl(file):
#         df = pd.read_json(file, orient = 'values', lines= True, dtype = str)
#         df.columns = regrid_fromraw_cols 
#         return df[regrid_toRAD_cols].rename(columns={'ll_uuid': 'rid'})
#     regrid = pool.map(func, list_of_files)
#     regrid = pd.concat(regrid, ignore_index=True)  
# regrid.shape # (3215105, 10)    
# print(sf_code, regrid.shape)    # MO SHOULD BE: 90sec, (3330480, 19)
# regrid.to_feather(f'data/regrid-feather/{sf_code}.ftr')    